# Model Evaluation & Visualization

This notebook allows you to visualize the performance of the trained Diabetes and Kidney Disease models using the training/validation datasets.

It reproduces the exact preprocessing steps used during training to ensure the metrics (Confusion Matrix, ROC Curve) are accurate representations of the model's performance on the validation split.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score

# Set Style
plt.style.use('default')
sns.set_theme(style="whitegrid")

# Paths
ML_PATH = "predictions/ml"
print(f"Using ML Path: {ML_PATH}")

## 1. Diabetes Model Evaluation

In [ ]:
# Load Data
df_diab = pd.read_csv(os.path.join(ML_PATH, "diabetes.csv"))

# Preprocessing (Imputation)
zero_impute_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in zero_impute_cols:
    if col in df_diab.columns:
        median_val = df_diab[df_diab[col] != 0][col].median()
        df_diab[col] = df_diab[col].replace(0, median_val)

# Select Features
X_d = df_diab[['Age', 'BMI', 'BloodPressure', 'Glucose']]
y_d = df_diab['Outcome']

# Data Split (Random State 15)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0.2, random_state=15)

# Load Model
model_d = joblib.load(os.path.join(ML_PATH, "diabetes_model.pkl"))
scaler_d = joblib.load(os.path.join(ML_PATH, "diab_scaler.pkl"))

# Predict
X_test_scaled_d = scaler_d.transform(X_test_d)
y_pred_d = model_d.predict(X_test_scaled_d)
y_prob_d = model_d.predict_proba(X_test_scaled_d)[:, 1]

# Metrics
acc_d = accuracy_score(y_test_d, y_pred_d)
print(f"Diabetes Test Accuracy: {acc_d:.2%}")
print(classification_report(y_test_d, y_pred_d))

In [ ]:
# Visualize Confusion Matrix
plt.figure(figsize=(6, 5))
cm_d = confusion_matrix(y_test_d, y_pred_d)
sns.heatmap(cm_d, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'], 
            yticklabels=['Negative', 'Positive'])
plt.title(f'Diabetes Confusion Matrix (Acc: {acc_d:.2%})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Visualize ROC Curve
fpr_d, tpr_d, _ = roc_curve(y_test_d, y_prob_d)
roc_auc_d = auc(fpr_d, tpr_d)

plt.figure(figsize=(6, 5))
plt.plot(fpr_d, tpr_d, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_d:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Diabetes ROC Curve')
plt.legend(loc="lower right")
plt.show()

## 2. Kidney Model Evaluation

In [ ]:
# Load Data
df_kid = pd.read_csv(os.path.join(ML_PATH, "kidney.csv"))

# Rename Columns
rename_map = {
    'Bp': 'Blood Pressure', 'Sg': 'Specific Gravity', 'Al': 'Albumin', 'Su': 'Sugar',
    'Rbc': 'Red Blood Cell', 'Bu': 'Urea', 'Sc': 'Creatinine', 'Sod': 'Sodium',
    'Pot': 'Pottasium', 'Hemo': 'Hemoglobin', 'Wbcc': 'White Blood Cell Count',
    'Rbcc': 'Red Blood Cell Count', 'Htn': 'Hypertension', 'Class': 'Predicted Class'
}
df_kid.rename(columns=rename_map, inplace=True)

# Filter & Clean (Winsorization)
df_kid = df_kid[df_kid['Pottasium'] <= 7]
df_kid = df_kid[df_kid['Hemoglobin'] <= 20]
df_kid = df_kid[df_kid['Blood Pressure'] >= 50]

df_kid['Creatinine'] = df_kid['Creatinine'].clip(0, 15)
df_kid['Pottasium'] = df_kid['Pottasium'].clip(2, 7)
df_kid['Hemoglobin'] = df_kid['Hemoglobin'].clip(4, 20)
df_kid['Sodium'] = df_kid['Sodium'].clip(100, 180)
df_kid['Blood Pressure'] = df_kid['Blood Pressure'].clip(60, 180)

# Select Features
req_cols = ['Predicted Class', 'Creatinine', 'Pottasium', 'Hemoglobin', 
            'Sodium', 'Blood Pressure', 'Red Blood Cell', 'Urea', 'Albumin']
df_kid = df_kid[req_cols]

X_k = df_kid.drop("Predicted Class", axis=1)
y_k = df_kid["Predicted Class"]

# Data Split (Random State 15)
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X_k, y_k, test_size=0.2, random_state=15)

# Load Model
model_k = joblib.load(os.path.join(ML_PATH, "kidney_model.pkl"))
scaler_k = joblib.load(os.path.join(ML_PATH, "kid_scaler.pkl"))

# Predict
X_test_scaled_k = scaler_k.transform(X_test_k)
y_pred_k = model_k.predict(X_test_scaled_k)
y_prob_k = model_k.predict_proba(X_test_scaled_k)[:, 1]

# Metrics
acc_k = accuracy_score(y_test_k, y_pred_k)
print(f"Kidney Test Accuracy: {acc_k:.2%}")
print(classification_report(y_test_k, y_pred_k))

In [ ]:
# Visualize Confusion Matrix
plt.figure(figsize=(6, 5))
cm_k = confusion_matrix(y_test_k, y_pred_k)
sns.heatmap(cm_k, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['No CKD', 'CKD'], 
            yticklabels=['No CKD', 'CKD'])
plt.title(f'Kidney Confusion Matrix (Acc: {acc_k:.2%})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Visualize ROC Curve
fpr_k, tpr_k, _ = roc_curve(y_test_k, y_prob_k)
roc_auc_k = auc(fpr_k, tpr_k)

plt.figure(figsize=(6, 5))
plt.plot(fpr_k, tpr_k, color='darkred', lw=2, label=f'ROC curve (area = {roc_auc_k:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Kidney ROC Curve')
plt.legend(loc="lower right")
plt.show()